## Data

In [36]:
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.options.display.max_colwidth=None
pd.options.display.max_rows=100

Using the cleaned data

In [37]:
df = pd.read_csv(r'D:\replies18june.csv')

C:\Users\lvshu\AppData\Local\Temp\ipykernel_13932\1055199631.py:1: DtypeWarning: Columns (35,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\replies18june.csv')


In [38]:
df_new = df.copy
df_new = df[['id_reply','id_original', 'to_tweetid_reply', 'user_id_reply', 'text_original', 'text_reply', 'pred_label']]
df_new = df_new.head(10) # This can be changed if we want to do more experiments

In [39]:
df_new

,id_reply,id_original,to_tweetid_reply,user_id_reply,text_original,text_reply,pred_label
0,1571540022062350336,1571539188402495488,1571539188402495488,1502656788520443904,"75% des Français se disent favorables à l’énergie #nucléaire.\n\nUn choix de raison, pour retrouver notre indépendance #énergétique et réussir notre transition #écologique.\n\nhttps://www.lejdd.fr/Politique/sondage-75-des-francais-se-disent-favorables-au-nucleaire-4134832",@charlesrodwell_ C’est nouveau ça chez #LREM de partager ce « choix de raison » non ?,non-abusive
1,1568293292256673536,1568162877181247488,1568162877181247488,1124410163895648384,"Il y a 70 ans, le 6 février 1952, sa Majesté la Reine accédait au trône britannique, à l’âge de 25 ans. \n\nVrai témoin historique, nous avons tous des souvenirs avec la reine Elizabeth II.\n\nProfonde tristesse. https://twitter.com/charlesrodwell_/status/1568162877181247490/photo/1",@charlesrodwell_ Euh...euh...quel genre de souvenirs ?,non-abusive
2,1568120472038000128,1567571556129280000,1567571556129280000,113150757,"Nous avons auditionné ce matin la société @vertmarine à la suite de sa décision de fermer 30 piscines publiques dont elle a la gestion. Avec @FdeMazieres, nous continuons de tout mettre en œuvre pour permettre la réouverture rapide de la piscine Montbauron de @Versailles 👇 https://twitter.com/charlesrodwell_/status/1567571556129280001/photo/1",@charlesrodwell_ Vous appartenez à une majorité qui a soutenu un exécutif qui a \nVendu Alstom énergie \nDésinvesti dans le nucléaire \nFermé Fessenheim \n\nVous n’avez que ce que vous méritez.,non-abusive
3,1575910131464015616,1575788225846968320,1575788225846968320,1573415131157086208,"Sciences Po, c’est l’école libre des sciences politiques.\n\nSon héritage et son devoir, c’est de faire vivre les grands débats qui animent la société française en cultivant notre indépendance culturelle.\n\nÀ cette emprise, nous ne pouvons nous résoudre. Il y a urgence à agir. « LEXPRESS: Race, genre, wokisme et... sorcières : Sciences Po Reims sous emprise américaine https://www.lexpress.fr/actualite/societe/race-genre-wokisme-et-sorcieres-sciences-po-reims-sous-emprise-americaine_2180938.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1664380735 https://twitter.com/LEXPRESS/status/1575192994239725589/photo/1 — https://twitter.com/lexpress/status/1575192994239725589 »",@charlesrodwell_ Oui c'est l'ecole de la supremacie civilisationnelle et politique de l'occident ! Marre de ces méchants noirs et musulmans qui veulent nous grand-remplacer et qui se réveillent pour défendre leurs droits ! Et pourquoi pas l'égalité tant qu'on y est ???,non-abusive
4,1576000140267360512,1575788225846968320,1575788225846968320,863898690209107840,"Sciences Po, c’est l’école libre des sciences politiques.\n\nSon héritage et son devoir, c’est de faire vivre les grands débats qui animent la société française en cultivant notre indépendance culturelle.\n\nÀ cette emprise, nous ne pouvons nous résoudre. Il y a urgence à agir. « LEXPRESS: Race, genre, wokisme et... sorcières : Sciences Po Reims sous emprise américaine https://www.lexpress.fr/actualite/societe/race-genre-wokisme-et-sorcieres-sciences-po-reims-sous-emprise-americaine_2180938.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1664380735 https://twitter.com/LEXPRESS/status/1575192994239725589/photo/1 — https://twitter.com/lexpress/status/1575192994239725589 »",@charlesrodwell_ Se plaindre de l'influence américaine sur le campus euro-américain c'est un peu comme se plaindre que l'eau mouille,non-abusive
5,1566709341855645440,1566451264254353408,1566451264254353408,1218447306061754368,"Mensonge, encore un !\n\n✅ L’Allemagne met en place une contribution obligatoire des énergéticiens… qui existe déjà en France ! et qui rapporte plusieurs milliards d’euros.\n\n❌ Le gouvernement allemand confirme qu’il n’y a pas de nouvelle taxe dans son nouveau paquet d’aides.","@charlesrodwell_ Vous êtes ridicule, arrê

## Ask LLM

In [40]:
from openai import OpenAI
import json
import re

client = OpenAI(
    api_key = "abc", # Change this to your OpenAI API key
)

In [41]:
def get_abusive_score(client, text_to_analyse, original_text):
    tweets_content = "\n".join([f"{i+1}. {tweet}" for i, tweet in enumerate(text_to_analyse)])
    
    messages = [
        {"role": "system", "content": "You are an annotator for the tweets in French."},
        {"role": "user", "content": (
            f"Now you are an annotator for the tweets which are the responses for French politicians. Below is an original tweet and its replies. "
            f"For each reply in the given list, considering the original tweet, annotate the reply as abusive or not abusive in the format of: "
            f"{{'index':'score'}} which can be saved into a json file directly (separating with ','). For the score, abusive replies should be labelled as 1, non-abusive replies should be labelled as 0.\n\n"
            f"When annotating the replies, please consider the following coding scheme:\n"
            f"Abusive: Negative, toxic, aggressive, threatening and demeaning comments that targets the MP as an individual (and not as a politician). These include:\n"
            f"1) slurs and hate speech (including censored terms): Direct and explicit words that are commonly classified as hateful towards an individual or group based on identity characteristics. "
            f"This also includes instances of the word being included in its censored version to get around automatic content moderation (for example p*** = “pute”)\n"
            f"2) aggressive or demeaning language: Insulting, aggressive, toxic, demeaning and belittling language targeting the MP as an individual (but not necessarily crossing the line to illegal hate speech).\n"
            f"3) Personal negative remarks: Any comment on personal characteristics that do not have anything to do with the person’s duties as politicians. These include comments on dress, personal appearance, voice, and other personal aspects not related to their duties as politician. Note that these comments can sometimes seem positive at face value (“tu es belle”) but be considered abusive in context as they aim to reduce the MP to their physical appearance.\n"
            f"4) Threats: Evoking threats of physical or sexual violence (directly or indirectly).\n"
            f"Attention, everything that is negative but explicitly targets the individual's behavior as a politician and their policies, rather than the individual as a person, should be labelled as 'non-abusive'. Pay attention to suspected cases of sarcasm: for example 'waouh il est intelligent........' is different from 'comme il est intelligent !👏'\n"
            f"Here is the original tweet:\n{original_text}\n\nHere are the replies:\n{tweets_content}\n\n"
            f"Please ensure that the output is a well-formed JSON object with double quotes around property names and values, NEVER return json file directly, only give me the strings."
        )}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.5,
    )
    
    response_message = response.choices[0].message.content

    response_message_clean = re.sub(r'[\n\t]', '', response_message)
    
    if not response_message_clean.startswith('{'):
        response_message_clean = '{' + response_message_clean
    if not response_message_clean.endswith('}'):
        response_message_clean = response_message_clean + '}'
    
    print(response_message_clean)
    
    try:
        annotations = json.loads(response_message_clean)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return None
    
    return annotations

In [42]:
grouped = df_new.groupby('to_tweetid_reply')

all_results = []

for tweet_id, group in grouped:
    original_text = group.iloc[0]['text_original']
    replies_texts = group['text_reply'].tolist()
    replies_ids = group['id_reply'].tolist()

    annotations = get_abusive_score(client, replies_texts, original_text)
    
    if annotations:
        for index, score in annotations.items():
            reply_index = int(index) - 1
            reply_text = replies_texts[reply_index]
            reply_id = replies_ids[reply_index]
            all_results.append([reply_id, original_text, reply_text, score])

results_df = pd.DataFrame(all_results, columns=['id_reply', 'original_tweet', 'reply', 'gpt_4_score'])

{  "1": 0}
{  "1": 0}
{  "1": "0"}
{  "1": 0}
{    "1": 0,    "2": 0,    "3": 0,    "4": 0}
{  "1": 0,  "2": 0}


In [43]:
results_df

,id_reply,original_tweet,reply,gpt_4_score
0,1566709341855645440,"Mensonge, encore un !\n\n✅ L’Allemagne met en place une contribution obligatoire des énergéticiens… qui existe déjà en France ! et qui rapporte plusieurs milliards d’euros.\n\n❌ Le gouvernement allemand confirme qu’il n’y a pas de nouvelle taxe dans son nouveau paquet d’aides.","@charlesrodwell_ Vous êtes ridicule, arrêter de jouer avec les mots pour masquer le fait que vous vous opposez à taxer les profiteurs de crise, qui en +, se gavent d'argent public et pire vous protégez ces profits via vos lois sur l'optimisation fiscale\nHonteux\n« attac_fr: [Thread] L'Allemagne a-t-elle décidé d'instaurer une taxe sur les #superprofits ?\n\n""Le gouvernement allemand n'a absolument pas annoncé une taxe sur les superprofits des énergéticiens"", a insisté Bercy dans un message envoyé aux journalistes ce dimanche.\nhttps://www.lefigaro.fr/conjoncture/superprofits-l-allemagne-n-a-pas-annonce-de-taxe-insiste-bercy-20220905 — https://twitter.com/attac_fr/status/1566699075134783489?t=niXJudo1MBxApqECkJ5Ppw&s=19 » https://twitter.com/WoobooDk/status/1566709341855645696/photo/1",0
1,1568120472038000128,"Nous avons auditionné ce matin la société @vertmarine à la suite de sa décision de fermer 30 piscines publiques dont elle a la gestion. Avec @FdeMazieres, nous continuons de tout mettre en œuvre pour permettre la réouverture rapide de la piscine Montbauron de @Versailles 👇 https://twitter.com/charlesrodwell_/status/1567571556129280001/photo/1",@charlesrodwell_ Vous appartenez à une majorité qui a soutenu un exécutif qui a \nVendu Alstom énergie \nDésinvesti dans le nucléaire \nFermé Fessenheim \n\nVous n’avez que ce que vous méritez.,0
2,1568293292256673536,"Il y a 70 ans, le 6 février 1952, sa Majesté la Reine accédait au trône britannique, à l’âge de 25 ans. \n\nVrai témoin historique, nous avons tous des souvenirs avec la reine Elizabeth II.\n\nProfonde tristesse. https://twitter.com/charlesrodwell_/status/1568162877181247490/photo/1",@charlesrodwell_ Euh...euh...quel genre de souvenirs ?,0
3,1571540022062350336,"75% des Français se disent favorables à l’énergie #nucléaire.\n\nUn choix de raison, pour retrouver notre indépendance #énergétique et réussir notre transition #écologique.\n\nhttps://www.lejdd.fr/Politique/sondage-75-des-francais-se-disent-favorables-au-nucleaire-4134832",@charlesrodwell_ C’est nouveau ça chez #LREM de partager ce « choix de raison » non ?,0
4,1574744090213637888,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous apprenait il y a quelques semaines qu’à Marseille 55% des personnes interpellées étaient étrangères. Comme à Paris et à Lyon. C’est ça la réalité ! » https://twitter.com/BallardPhilippe/status/1574700793659240448/video/1,@BallardPhilippe Le grand remplacement,0
5,1574998199433302016,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous apprenait il y a quelques semaines qu’à Marseille 55% des personnes interpellées étaient étrangères. Comme à Paris et à Lyon. C’est ça la réalité ! » https://twitter.com/BallardPhilippe/status/1574700793659240448/video/1,@BallardPhilippe Et 45% ou presque restant d'origine étrangère ne l'oubliez pas aussi....,0
6,1574701582003962112,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous apprenait il y a quelques semaines qu’à Marseille 55% des personnes interpellées étaient étrangères. Comme à Paris et à Lyon. C’est ça la réalité ! » https://twitter.com/BallardPhilippe/status/1574700793659240448/video/1,@BallardPhilippe Déjà faire la réunification avec la Bretagne historique géographique et culturelle!!\n\nOccupez vous plutôt de votre sinistre région parisienne!!\n\n44=BZH,0
7,1574753413392470016,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous

In [44]:
merged_df = pd.merge(results_df, df_new[['id_reply', 'pred_label']], on='id_reply', how='left')

In [45]:
merged_df

,id_reply,original_tweet,reply,gpt_4_score,pred_label
0,1566709341855645440,"Mensonge, encore un !\n\n✅ L’Allemagne met en place une contribution obligatoire des énergéticiens… qui existe déjà en France ! et qui rapporte plusieurs milliards d’euros.\n\n❌ Le gouvernement allemand confirme qu’il n’y a pas de nouvelle taxe dans son nouveau paquet d’aides.","@charlesrodwell_ Vous êtes ridicule, arrêter de jouer avec les mots pour masquer le fait que vous vous opposez à taxer les profiteurs de crise, qui en +, se gavent d'argent public et pire vous protégez ces profits via vos lois sur l'optimisation fiscale\nHonteux\n« attac_fr: [Thread] L'Allemagne a-t-elle décidé d'instaurer une taxe sur les #superprofits ?\n\n""Le gouvernement allemand n'a absolument pas annoncé une taxe sur les superprofits des énergéticiens"", a insisté Bercy dans un message envoyé aux journalistes ce dimanche.\nhttps://www.lefigaro.fr/conjoncture/superprofits-l-allemagne-n-a-pas-annonce-de-taxe-insiste-bercy-20220905 — https://twitter.com/attac_fr/status/1566699075134783489?t=niXJudo1MBxApqECkJ5Ppw&s=19 » https://twitter.com/WoobooDk/status/1566709341855645696/photo/1",0,non-abusive
1,1568120472038000128,"Nous avons auditionné ce matin la société @vertmarine à la suite de sa décision de fermer 30 piscines publiques dont elle a la gestion. Avec @FdeMazieres, nous continuons de tout mettre en œuvre pour permettre la réouverture rapide de la piscine Montbauron de @Versailles 👇 https://twitter.com/charlesrodwell_/status/1567571556129280001/photo/1",@charlesrodwell_ Vous appartenez à une majorité qui a soutenu un exécutif qui a \nVendu Alstom énergie \nDésinvesti dans le nucléaire \nFermé Fessenheim \n\nVous n’avez que ce que vous méritez.,0,non-abusive
2,1568293292256673536,"Il y a 70 ans, le 6 février 1952, sa Majesté la Reine accédait au trône britannique, à l’âge de 25 ans. \n\nVrai témoin historique, nous avons tous des souvenirs avec la reine Elizabeth II.\n\nProfonde tristesse. https://twitter.com/charlesrodwell_/status/1568162877181247490/photo/1",@charlesrodwell_ Euh...euh...quel genre de souvenirs ?,0,non-abusive
3,1571540022062350336,"75% des Français se disent favorables à l’énergie #nucléaire.\n\nUn choix de raison, pour retrouver notre indépendance #énergétique et réussir notre transition #écologique.\n\nhttps://www.lejdd.fr/Politique/sondage-75-des-francais-se-disent-favorables-au-nucleaire-4134832",@charlesrodwell_ C’est nouveau ça chez #LREM de partager ce « choix de raison » non ?,0,non-abusive
4,1574744090213637888,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous apprenait il y a quelques semaines qu’à Marseille 55% des personnes interpellées étaient étrangères. Comme à Paris et à Lyon. C’est ça la réalité ! » https://twitter.com/BallardPhilippe/status/1574700793659240448/video/1,@BallardPhilippe Le grand remplacement,0,non-abusive
5,1574998199433302016,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous apprenait il y a quelques semaines qu’à Marseille 55% des personnes interpellées étaient étrangères. Comme à Paris et à Lyon. C’est ça la réalité ! » https://twitter.com/BallardPhilippe/status/1574700793659240448/video/1,@BallardPhilippe Et 45% ou presque restant d'origine étrangère ne l'oubliez pas aussi....,0,non-abusive
6,1574701582003962112,📺 « Nantes est devenue une poudrière parce qu’on est d’un laxisme total en matière d’immigration ! \n\nM. Darmanin nous apprenait il y a quelques semaines qu’à Marseille 55% des personnes interpellées étaient étrangères. Comme à Paris et à Lyon. C’est ça la réalité ! » https://twitter.com/BallardPhilippe/status/1574700793659240448/video/1,@BallardPhilippe Déjà faire la réunification avec la Bretagne historique géographique et culturelle!!\n\nOccupez vous plutôt de votre sinistre région parisienne!!\n\n44=BZH,0,non-abusive
7,1574753413392470016,📺 « Nantes est devenue u

In [46]:
merged_df.to_csv('with_context_gpt_4_test.csv', index=False)

Notes: 

Here I managed to use a toy dataset (only 10 examples) to do the test: asking gpt-4o to classify the tweets as abusive or non-abusive ones with their corresponding context. 
The idea is: using 'to_tweetid_reply'to group by the replies for the same original tweet; iterating through each group including extracting the original tweet and collecting replies and their IDs, giving both context and replies to gpt-4o to get the score; converting the replies in text version by retrieving index and generating a new df; after that merging this df with the "pred_label" from original dataset so that we can easily make a comparison. 

This is a script for testing, and the usage is really easy: inputing a selected dataset directly, and the script will return you a csv file with the score provided by LLM for which the context has been considered. 